In [1]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()

    for i in range(num_gpus):
        print(f"GPU {i}:")
        gpu_properties = torch.cuda.get_device_properties(i)

        # Total memory
        total_memory = gpu_properties.total_memory / 1024**2  # Convert to MB
        print(f"  Total Memory: {total_memory:.2f} MB")

        # Current memory usage
        allocated_memory = torch.cuda.memory_allocated(i) / 1024**2  # Convert to MB
        print(f"  Allocated Memory: {allocated_memory:.2f} MB")

        # Cached memory
        cached_memory = torch.cuda.memory_reserved(i) / 1024**2  # Convert to MB
        print(f"  Cached Memory: {cached_memory:.2f} MB")

        print("--------------------------------------------------")
else:
    print("CUDA is not available. No GPU detected.")

print(torch.tensor([1.0, 2.0], device='cuda'))

True
1
0
NVIDIA A100-SXM4-80GB
12.1
GPU 0:
  Total Memory: 81092.56 MB
  Allocated Memory: 0.00 MB
  Cached Memory: 0.00 MB
--------------------------------------------------
tensor([1., 2.], device='cuda:0')


In [2]:
!nvidia-smi
!nvcc --version

Fri Dec  1 19:32:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB           On | 00000000:46:00.0 Off |                    0 |
| N/A   29C    P0               67W / 400W|    877MiB / 81920MiB |     13%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import torch

from textwrap import fill
from IPython.display import Markdown

from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain import HuggingFacePipeline

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')
NUM_CHAPTERS = 10
MAIN_NAME = "Klaus"

In [4]:
MODEL_NAME = 'Intel/neural-chat-7b-v3-1'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    use_flash_attention_2=False,
    trust_remote_code=True,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 5000
#generation_config.min_new_tokens = -1
#generation_config.max_length = 10000
#generation_config.top_p = 0.9
#generation_config.top_k = 50
#generation_config.do_sample = True
#generation_config.temperature = 0.7
generation_config.repetition_penalty = 1.3

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
pipeline.model.config.pad_token_id = pipeline.model.config.eos_token_id

In [6]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [7]:
longchat_template = f"""### System: 
Objective: Create a {NUM_CHAPTERS}-chapter children's book set in a magical Korean world with dragons. Each 1000-word chapter features a unique, open-ended adventure.
AI's Role:
Craft engaging, informative stories for young readers.
Use a friendly tone, rich descriptions, diverse characters.
Focus on positive themes like friendship and learning.
Mix in humor and educational elements.
Build resilience and empathy through challenges.
End stories with hope or lessons, avoiding negativity.
Process: Write one detailed chapter at a time, awaiting user prompts for subsequent chapters.

Prologue: The Quest for Harmony in Korea's Enchanted Realms

In a realm where ancient mystique and modern marvels intertwine, an epic saga awaits to be told. Here, in the heart of Korea's enchanted lands, a world brimming with magic and mystery beckons. A world where dragons trace majestic arcs across the sky and mythical beings whisper the secrets of the ages. 
In this realm of wonder, two heroes are predestined to unite: {MAIN_NAME}, the resplendent phoenix from the soaring cliffs of Pohang, a creature of fire and lore, and Nubzuki, the inquisitive platfish from the tranquil waters of Daejeon, wise and whimsical.
Their journey, a tapestry of peril and enlightenment, will sweep them through realms where ancient trees murmur forgotten tales, across vast meadows echoing with celestial music, and into vibrant cities where history and future meld in perfect harmony.
This odyssey, however, is far from a mere escapade; it's a quest of discovery, resilience, and kinship. Through trials and tribulations, {MAIN_NAME} and Nubzuki will delve deep into the heart of Korean culture, encountering its rich traditions, arts, and ancient wisdoms, each step a lesson in unity and strength.
But lurking in the shadows is a darkness, a force that seeks to unbalance this world of wonder. Our heroes, {MAIN_NAME} and Nubzuki, must stand as one against this burgeoning chaos. In their quest, they will learn that true might blossoms from togetherness and that courage can be found in the most unexpected places.
So, brace yourselves, dear readers, for an adventure beyond compare. Soar with {MAIN_NAME} into realms uncharted, dive with Nubzuki into the depths of the extraordinary. Together, we will traverse a world where Korean lore comes alive, where each twist and turn is a doorway to the extraordinary.
Prepare to embark on "The Quest for Harmony in Korea's Enchanted Realms," a narrative pulsating with magic, brimming with lessons, and shimmering with the light of enduring hope.
"""
longchat_template.format(MAIN_NAME = MAIN_NAME, NUM_CHAPTERS = NUM_CHAPTERS)
longchat_template += """
{history}
### User: 
{input}
### Assistant:
"""

In [8]:
longchat_prompt_template = PromptTemplate(
    input_variables=["history", "input"], template=longchat_template
)

In [9]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(ai_prefix="Assistant", human_prefix="User"),
    prompt=longchat_prompt_template,
)

In [10]:
print(conversation_buf("Write now only the first chapter!")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter One: Finding Friendships Amidst Fireflies

Klaus was not always aware he had wings. He thought his life revolved around nothing more than being a guardian of the sacred Phoenix Flame - a flame which protected all living creatures within these mysterious lands. His days were spent watching over the glowing orb, ensuring no harm came near it. But little did he know, there existed another world outside those walls, waiting just beyond the horizon. It would take meeting someone very special before he could truly understand what lay ahead...

Nearby, nestled among verdant hillsides and serene forests, lived Nubzuki – a curious Platypus who loved exploring every nook and cranny of her home. She adored discovering new things about nature, especially when she stumbled upon something unusual. Her favorite pastime involved observing how different animals interacted with one another, fascinated by the intricacies of friendships formed between species. Little did she realize, though, that

In [11]:
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Option 1 (Challenge Acceptance): After spending some time getting acquainted, Klaus shares his responsibility as a protector of the Phoenix Flame. Impressed by his dedication, Nubzuki offers to help safeguard the precious treasure. This marks the beginning of their partnership, filled with adventures and camaraderie.

Option 2 (Uncovering Secrets): While flying amongst the fireflies, our duo stumbles upon a hidden cave containing artifacts related to long lost legends. Excitedly, they decide to investigate further, hoping to unlock mysteries surrounding the origins of their respective worlds.

Option 3 (Encountering New Friends): During their exploration, Klaus and Nubzuki meet various fantastical creatures such as fairy folk, dragon hatchlings, and even a group of talking plants. These encounters lead to exciting collaborations and lifelong bonds, expanding their circle of friends throughout the land.

Option 4 (Facing Dark Forces): Unbeknownst to them, dark forces seek to disrupt th

In [12]:
import random
random.seed()
num_dec = 4  # number of decisions

decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 2nd chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Two: Into the Depths of Mystery

With renewed determination after sharing their individual responsibilities, Klaus and Nubzuki ventured forth into unexplored territories. Their eyes sparkled with anticipation as they entered the secret chamber concealed behind the waterfall. Inside, they encountered a breathtaking sight—a collection of relics dating back centuries ago, preserved beneath layers of dust and cobwebs. Among these treasures stood a magnificent statue depicting a winged figure holding aloft a fiery globe. Its presence seemed familiar yet elusive, stirring memories buried deep inside their hearts.

Curiosity piqued, they carefully examined the statuesque form, tracing the etched symbols carved onto its surface. Gradually, they deciphered cryptic messages hinting toward a powerful connection between the Phoenix Flame and the legendary Dragon King. Could this be the key to understanding why they crossed paths? Was it destiny that brought them together?

Excitement surg

In [13]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 3rd chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Three: Whispers of Ancient Wisdom

Guided by the warmth of their newly acquired knowledge, Klaus and Nubzuki sought solace in the company of elder figures whose experiences spanned generations. From the wisened sage, Kwan Yin, they learned the ways of compassionate leadership and selfless service. Understanding the value of humility, they embraced their role as protectors without losing touch with their innate humanity.

From Master Lee, a skilled martial artist, they gained insight into the discipline required to master their powers. Balancing grace and agility became second nature, allowing them to navigate dangerous situations swiftly and effectively.

They also met Grandmother Moon, a gifted seeress known for her profound intuition and spiritual healing practices. Learning the language of dreams allowed them to connect deeply with their subconscious minds, revealing truths otherwise obscured by everyday distractions.

Throughout their training sessions, Klaus and Nubzuki g

In [14]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 4th chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Four: Celebrating Life's Wonders

Having faced numerous tests and challenges, Klaus and Nubzuki emerged victorious, bringing balance and harmony back to their world. Now, they turned their attention to spreading awareness of the importance of cultural heritage and tradition. Traveling across the seven provinces, they visited temples, palaces, and villages alike, sharing stories of their remarkable journey and encouraging everyone to celebrate the uniqueness of their customs.

During their visits, they witnessed firsthand the transformational impact of preserving local folklore and rituals. Children laughed and played games inspired by ancient fables, adults gathered around bonfires retelling timeless tales, and elderly citizens passed on cherished recipes handed down through countless generations. Everywhere they went, they saw communities thriving because of their commitment to maintaining their roots.

At last, they reached the capital city, where they joined hands with lead

In [15]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 5th chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Five: Legacy of Love

Time marches relentlessly onward, carrying Klaus and Nubzuki into the sunset years of their lives. Despite their advancing age, their passion for adventure remained undeterred. They took comfort in knowing that their efforts had laid the foundation for a harmonious society built on inclusivity and acceptance.

Now, instead of focusing solely on protection and conservation, they shifted focus to education and empowerment. Establishing institutions aimed at cultivating the next generation of visionary thinkers, innovators, and dreamers, they ensured that the principles instilled in them would never die.

Under their tutelage, aspiring artists explored the boundlessness of creativity, scientists probed the depths of scientific advancements, and philosophers contemplated the meaning of human existence. Students from all walks of life converged in classrooms, libraries, laboratories, and studios, exchanging ideas and challenging conventional thinking.

Oversee

In [16]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 6th chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Six: Spiritual Ascension

After bidding farewell to their closest companions, Klaus and Nubzuki ascend into the heavens, guided by the radiant constellations overhead. Surrounded by the ethereal beauty of the galaxy, they reflected on their lives, grateful for the opportunities afforded to them.

Soon, they arrived at the gates of heaven, greeted by angelic hosts welcoming them with open arms. Eager to reunite with their dearly departed family members, they traversed the pearly streets, awash with divine energy.

Here, they found solace in the eternal embrace of their parents, siblings, mentors, and teachers. Laughing and crying together, they recounted cherished moments from their earthbound lives, realizing that their relationships extended far beyond the confines of flesh and blood.

Eventually, they settled into their new abode, taking delight in the simple pleasures of communicating telepathically, meditating in gardens bursting with fragrant flowers, and partaking in fea

In [17]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 7th chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Seven: Universal Guardianship

Centuries passed, and the galaxy evolved rapidly due to technological advances and unprecedented levels of communication. Intergalactic trade routes opened, facilitated by advanced transportation systems capable of traveling faster than the speed of light. Civilization boomed exponentially, giving rise to innovative architectural structures, groundbreaking medical breakthroughs, and revolutionary artistic movements.

Amidst this era of rapid expansion, Klaus and Nubzuki observed closely, monitoring developments and assessing whether the core tenets of unity and love remained intact. Satisfied with the overall direction of societal evolution, they focused their attentions elsewhere.

Turning their gaze to less developed planets, they identified emerging conflicts and disputes requiring intervention. Utilizing their extensive network of contacts, they assembled a coalition comprising representatives from various star systems willing to contribute r

In [18]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 8th chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Eight: Final Battle

Dark clouds amassed ominously on the periphery of the galaxy, signalling the arrival of a menacing entity hellbent on annihilation. Led by a vile despot driven mad by greed and lust for power, this merciless army threatened to destroy everything Klaus and Nubzuki had worked so hard to achieve.

Realizing the severity of the situation, they convened with their trusted advisors, strategizing methods to counteract the approaching invasion. Reinforcements poured in from neighboring solar systems, pledging allegiance to the cause of righteousness.

Raising shields of defense and deploying fleets of ships armed with state-of-the-art technology, they engaged the hostile fleet in a series of intense dogfight engagements. Laser barrages pierced through armor plating, missiles exploded midair, and fighter pilots executed death-defying acrobats.

Meanwhile, Klaus and Nubzuki utilized their exceptional mental faculties to manipulate the flow of battle, directing reinf

In [19]:
decision = random.randint(1, num_dec)
print(f"Decision: {decision}")
print(conversation_buf(f"Write now the 9th chapter of {NUM_CHAPTERS} with using decision {decision}.")["response"])
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(conversation_buf(f"Create now 4 [1,2,3,4] decisions how the story could continue.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: 4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Nine: Immersion in Education

Recognizing the crucial role education plays in shaping the character and beliefs of youth, Klaus and Nubzuki invested considerable effort in developing curriculum tailored specifically to inculcate essential traits of unity and love. Combining theoretical instruction with practical application, they encouraged experiential learning, allowing students to apply concepts in real-world scenarios.

Schools sprung up everywhere, catering to the needs of diverse student demographics. Classroom settings varied widely, ranging from virtual environments accessed remotely to fully immersive simulations utilizing augmented reality devices. Teachers employed gamification techniques to keep students actively engaged, making learning fun and enjoyable.

Students participated in projects addressing social inequities, climate change mitigation, and international affairs. Collaborating with peers from varying backgrounds, they practised effective communication, co

In [20]:
decision = "c"
print(conversation_buf(f"Write now the 10th chapter with using decision {decision}. Find a happy or sad ending based on the previous decisions.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Chapter Ten: Epilogue

As the pages of time turned, the galaxy continued to grow and evolve, shaped by the choices of its residents. Decisions taken today influenced tomorrow's trajectory, and the rippling effects of Klaus and Nubzuki's actions reverberated throughout the cosmos.

Some chose to follow the footsteps of their inspirational predecessors, dedicating their lives to the promotion of unity and love. Others opted for alternative paths, carving out niches suited to their particular skill sets and desires. Still, others drifted apart, drawn by the magnetic pull of divergent goals and priorities.

Regardless of the directions they took, the galaxy remained a dynamic, ever-expanding playground for experimentation and growth. Innovations in technology, architecture, and philosophy pushed the envelope of human capability, constantly testing the boundaries of what was possible.

While some aspects of the galaxy retained vestiges of strife and discord, the majority of its population 

In [21]:
print(conversation_buf(f"Write now a detailed text to evaluate the decisions of {MAIN_NAME}, and show what was good or bad and why! Please call {MAIN_NAME} by you.")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Evaluation of Klaus' Decisions: Navigating the Path Towards Unity and Love

Klaus, a central protagonist in the story, demonstrated tremendous conviction and drive in his pursuit of unity and love. Born into a tumultuous world fraught with division and suffering, he quickly grasped the gravity of the circumstances and seized control of his destiny.

His initial decision to train himself in the art of fighting was motivated by a desire to protect his people from external threats. At the time, survival necessitated proficiency in combat, and Klaus excelled in this area. Later, however, he realized that violence alone wouldn't suffice in securing a stable future.

Instead, he channeled his warrior spirit into constructive activities, leveraging his military prowess to organize resistance against oppressive rulership. This shift in perspective enabled him to align with likeminded individuals, forming coalitions that would go on to shape the course of history.

Once freed from the yoke of 

In [22]:
print(conversation_buf.memory.buffer)

User: Write now only the first chapter!
Assistant:  Chapter One: Finding Friendships Amidst Fireflies

Klaus was not always aware he had wings. He thought his life revolved around nothing more than being a guardian of the sacred Phoenix Flame - a flame which protected all living creatures within these mysterious lands. His days were spent watching over the glowing orb, ensuring no harm came near it. But little did he know, there existed another world outside those walls, waiting just beyond the horizon. It would take meeting someone very special before he could truly understand what lay ahead...

Nearby, nestled among verdant hillsides and serene forests, lived Nubzuki – a curious Platypus who loved exploring every nook and cranny of her home. She adored discovering new things about nature, especially when she stumbled upon something unusual. Her favorite pastime involved observing how different animals interacted with one another, fascinated by the intricacies of friendships formed be